<h2 align="center">DLOps - Deployment</h2>

Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro

Neste script vamos aprender a desplegar (Deployment) nossos modelos em produção. Para isso usaremos [FastAPI](https://fastapi.tiangolo.com/), um *framework* de Python para o desenvolvimento de APIs, [Docker](https://www.docker.com/) para empacotar a API e  [Heroku](https://www.heroku.com/) para desplegar a PAI em produção. 

# FastAPI

Sabemos que existe diferentes frameworks para criar APIs em Python, como `Flask` por exemplo, aqui vamos a usar `FastAPI`, já que oferece muitas funcionalidades que usaremos ao longo desta série de posts. Se não conheces este projeto, te recomendo que navegar por sua [Documentação](https://fastapi.tiangolo.com/tutorial/).


> Podes instalar FastAPI com o comando `pip install fastapi[all]`.

Uma vez instalado, cria um script chamado `app.py` com o seguinte conteúdo:


```
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}
```

Agora, podemos começar tua API em local com o comando `uvicorn app:app --reload`. Se todo va bem, deverias ver um mensaje similar a:

![](sen_3.png)

Se agora abres teu navegador e digitas `http://localhost:8000/`, deverás ver a mensagem retornada pela API.

```
{
  "message": "Hello World"
}
```

Sencillo, ¿verdad? Ahora simplemente tenemos que hacer que esta API (que como puedes ver no es más que un script de Python) carge nuestro modelo, reciba entradas (en nuestro caso imágenes) y devuelva las predicciones. Esto lo conseguiremos con el siguiente código:


```
from fastapi import FastAPI, File, UploadFile
from PIL import Image
import onnxruntime as ort 
import numpy as np
import io
import math

app = FastAPI()

ort_session = ort.InferenceSession('models/binary_classifier_3.onnx')
TRHESHOLD = 0.5

@app.get("/")
async def root():
    return {"message": "Hello World"}

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    request_object_content = await file.read()
    img = Image.open(io.BytesIO(request_object_content))
    input = np.expand_dims(np.array(img, dtype=np.uint8), axis=0)
    ort_inputs = {
        "input": input
    }
    ort_output = ort_session.run(['output'], ort_inputs)[0]
    output = sigmoid(ort_output)
    return {
        "proba": output,
        "label": "3" if output > TRHESHOLD else "no 3"
    }
```

Una de las ventajas que FastAPI ofrece es la de generación automática de documentación interactiva. Si visitas `http://localhost:8000/docs`, podrás probar tu nuevo *endpoint* al cual enviarle imágenes y recibir las predicciones del modelo. Siéntete libre de personalizar tu API a tu gusto 😁.

# Docker